In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
import joblib

In [2]:
dataset = 'Asset15k.xlsx'
df = pd.read_excel(dataset)

In [3]:
df

,AssetType,SerialNo,Manufacturer,ModelNbr,Refrigerant,RefrigerantQty
0,FIRE DAMPER,NaN,NaN,NaN,NaN,NaN
1,FIRE DAMPER,TBA,NaN,TBA,NaN,0.0
2,SWITCHBOARD MECHANICAL,TBA,NaN,TBA,NaN,NaN
3,FIRE DAMPER,NaN,NaN,NaN,NaN,NaN
4,AIR DAMPER,TBA,NaN,TBA,NaN,NaN
...,...,...,...,...,...,...
14470,AIR HANDLING UNIT,TBA,COOLAIR,ASH 2000 FMSP,CHILLED WATER,0.0
14471,FIRE PANEL-GRAPHICS SYSTEM,TBA,NaN,TBA,R410A,0.0
14472,AIR HANDLING UNIT,TBA,COOLAIR,ASH 1500 FMSP,CHILLED WATER,0.0
14473,FIRE PANEL-GRAPHICS SYSTEM,TBA,NaN,TBA,R410A,0.0


In [4]:
X = df.iloc[:,1:-2]
y = df['AssetType']

In [5]:
X.head()

,SerialNo,Manufacturer,ModelNbr
0,NaN,NaN,NaN
1,TBA,NaN,TBA
2,TBA,NaN,TBA
3,NaN,NaN,NaN
4,TBA,NaN,TBA


In [6]:
y

0                       FIRE DAMPER
1                       FIRE DAMPER
2            SWITCHBOARD MECHANICAL
3                       FIRE DAMPER
4                        AIR DAMPER
                    ...            
14470             AIR HANDLING UNIT
14471    FIRE PANEL-GRAPHICS SYSTEM
14472             AIR HANDLING UNIT
14473    FIRE PANEL-GRAPHICS SYSTEM
14474             AIR HANDLING UNIT
Name: AssetType, Length: 14475, dtype: object

In [7]:
# Impute missing values
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [8]:
X = X_imputed

In [9]:
special_values = [1234, 'UNKOWN', 'Unknown', 'NA', 'NULL', 'UNKNOWN', 'TBA', 'N/A', 'NOT VISIBLE', '123TEST', 'UNABLE TO LOCATE', 'NO ID', 'NO ACCESS', 'UNKOWN', 'NaN', 'na', 'AS PER PICS']

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
X_copy = X.copy()

# Replace values using .loc to avoid SettingWithCopyWarning
X_copy.loc[:, 'SerialNo'] = X_copy['SerialNo'].replace(special_values, pd.NA)
X_copy.loc[:, 'Manufacturer'] = X_copy['Manufacturer'].replace(special_values, pd.NA)
X_copy.loc[:, 'ModelNbr'] = X_copy['ModelNbr'].replace(special_values, pd.NA)

# Now X_copy contains the modified DataFrame
X = X_copy

In [10]:
OE_X = joblib.load('model/OE_X')

In [11]:
LE_Y = joblib.load('model/LE_Y')

In [12]:
X = OE_X.transform(X.astype(str))

In [13]:
X

array([[76031.,   394.,  7364.],
       [76031.,   394.,  7364.],
       [76031.,   394.,  7364.],
       ...,
       [76031.,   259., 13792.],
       [76031.,   394.,  7364.],
       [76031.,   259., 13779.]])

In [14]:
y = LE_Y.transform(y.astype(str))

In [17]:
y

array([264, 264, 501, ...,  20, 285,  20])

In [15]:
rf = joblib.load('model/rf_final2_ordinal')

In [16]:
y_pred_encoded = rf.predict(X)

In [18]:
accuracy = accuracy_score(y,y_pred_encoded)
accuracy

0.609119170984456

In [23]:
X2 = pd.DataFrame({
    'SerialNo': ['TBA'],
    'Manufacturer': ['XFGST'],
    'ModelNbr': ['892GA']
})

In [25]:
X2

,SerialNo,Manufacturer,ModelNbr
0,TBA,XFGST,892GA


In [28]:
special_values = [1234, 'UNKOWN', 'Unknown', 'NA', 'NULL', 'UNKNOWN', 'TBA', 'N/A', 'NOT VISIBLE', '123TEST', 'UNABLE TO LOCATE', 'NO ID', 'NO ACCESS', 'UNKOWN', 'NaN', 'na', 'AS PER PICS']

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
X2_copy = X2.copy()

# Replace values using .loc to avoid SettingWithCopyWarning
X2_copy.loc[:, 'SerialNo'] = X2_copy['SerialNo'].replace(special_values, pd.NA)
X2_copy.loc[:, 'Manufacturer'] = X2_copy['Manufacturer'].replace(special_values, pd.NA)
X2_copy.loc[:, 'ModelNbr'] = X2_copy['ModelNbr'].replace(special_values, pd.NA)

# Now X_copy contains the modified DataFrame
X2 = X2_copy

In [29]:
X2 = OE_X.transform(X2)

In [30]:
y2 = rf.predict(X2)

In [31]:
y2

array([526])

In [32]:
y2_actual = LE_Y.inverse_transform(y2)
y2_actual

array(['VARIABLE SPEED DRIVE'], dtype=object)